In [54]:
import pandas as pd
from shapely.geometry import Point, MultiPoint
import osmnx as ox
from shapely import Polygon
import geojson
from geopandas import GeoDataFrame
import networkx as nx
import pickle

In [52]:
class stop_object:
    def __init__ (self, routes, connected_stops, all_walk_times):
        self.routes = routes
        self.connected_stops = connected_stops
        self.all_walk_times = all_walk_times

# Route object

#For each stop a route object
# Departure time, sequence, tripID

class stop_route:
    def __init__ (self, departures):
        self.departures = departures

In [47]:
gtfs_data = 'Input Data/study_area_gtfs_bus/'

# gtfs_calendar = pd.read_csv(gtfs_data+'calendar.txt', sep=",", header=0)
# gtfs_calendar_dates = pd.read_csv(gtfs_data+'calendar_dates.txt', sep=",", header=0)
# gtfs_routes = pd.read_csv(gtfs_data+'routes.txt', sep=",", header=0)
gtfs_stop_times = pd.read_csv(gtfs_data+'stop_times.txt', sep=",", header=0, dtype={3:str,5:str})
gtfs_stops = pd.read_csv(gtfs_data+'stops.txt', sep=",", header=0)
gtfs_trips = pd.read_csv(gtfs_data+'trips.txt', sep=",", header=0)

In [17]:
bounding_box = tuple([-1.647263,53.755816,-1.426849,53.847666])

point1 = (-1.647263,53.755816)  # Replace longitude1 and latitude1 with your actual values
point2 = (-1.426849,53.847666)  # Replace longitude2 and latitude2 with your actual values

# Create the bounding box polygon
bbox_polygon = Polygon([
    (point1[0], point1[1]),
    (point2[0], point1[1]),
    (point2[0], point2[1]),
    (point1[0], point2[1]),
    (point1[0], point1[1])
])

# Convert the polygon to GeoJSON format
geojson_polygon = geojson.Feature(geometry=bbox_polygon, properties={})

# Output the GeoJSON to a file
with open("Outputs/bounding_box.geojson", "w") as f:
    geojson.dump(geojson_polygon, f)

In [19]:
stop_geoms = {}

for i,r in gtfs_stops.iterrows():
    stop_geom = Point([r['stop_lon'],r['stop_lat']])
    if bbox_polygon.contains(stop_geom):
        stop_geoms[r['stop_id']] = Point([r['stop_lon'],r['stop_lat']])

In [21]:
multi_point = MultiPoint(list(stop_geoms.values()))

# Convert the MultiPoint to GeoJSON format
geojson_multipoint = geojson.Feature(geometry=multi_point, properties={})

# Output the GeoJSON to a file
with open("Outputs/stops_in_study.geojson", "w") as f:
    geojson.dump(geojson_multipoint, f)
    
G = ox.graph_from_bbox(53.847666,53.755816, -1.426849, -1.647263, network_type = 'walk')

edges = ox.graph_to_gdfs(G,nodes = False)

# Output to file
edge_attributes_gdf = GeoDataFrame(edges[['geometry']], crs="EPSG:4326", geometry=edges['geometry'])
edge_attributes_gdf.to_file("Outputs/network.json", driver="GeoJSON")

In [28]:
stop_to_node = {}
for stop in list(stop_geoms.keys()):
    next_stop = stop_geoms[stop]
    stop_to_node[stop] = ox.nearest_nodes(G, next_stop.x, next_stop.y)

In [45]:
mph = 2.98
mps = mph * 0.44704

stops_in_area = list(stop_to_node.keys())
stop_to_stop_walk_time = {}
num_stops = len(stops_in_area)

count_stops_processed = 0

for u in stops_in_area:
    u_node = [stop_to_node[u]]
    length, path = nx.multi_source_dijkstra(G, u_node,  weight='length')

    stop_to_stop_walk_time[u] = {}

    v = stops_in_area[0]

    for v in stops_in_area:
        stop_to_stop_walk_time[u][v] = (length[stop_to_node[v]] / mps)
    
    count_stops_processed += 1
    
    print('-------------------------------------')
    print('Overall Progress {} of {}'.format(count_stops_processed,num_stops))
    print('-------------------------------------')

-------------------------------------
Overall Progress 1 of 2095
-------------------------------------
-------------------------------------
Overall Progress 2 of 2095
-------------------------------------
-------------------------------------
Overall Progress 3 of 2095
-------------------------------------
-------------------------------------
Overall Progress 4 of 2095
-------------------------------------
-------------------------------------
Overall Progress 5 of 2095
-------------------------------------
-------------------------------------
Overall Progress 6 of 2095
-------------------------------------
-------------------------------------
Overall Progress 7 of 2095
-------------------------------------
-------------------------------------
Overall Progress 8 of 2095
-------------------------------------
-------------------------------------
Overall Progress 9 of 2095
-------------------------------------
-------------------------------------
Overall Progress 10 of 2095
-------

In [55]:
acceptable_walk_time = 450
start_index = 0
end_index = 2
stop_classes = {}
num_processed = 0

for stop_id in stops_in_area:
    filtered_stops = {stop: time for stop, time in stop_to_stop_walk_time[stop_id].items() if time < acceptable_walk_time}
    connecting_stops = list(filtered_stops.keys())
    stop_time = gtfs_stop_times[gtfs_stop_times['stop_id'].isin(connecting_stops)]
    stop_time = stop_time.merge(gtfs_trips[['trip_id','route_id']],left_on='trip_id',right_on='trip_id')

    route_dict = {}
    for route in list(set(list(stop_time['route_id']))):
        route_stop_times = stop_time[stop_time['route_id'] == route]
        route_stop_times = route_stop_times.drop_duplicates(subset=['arrival_time'])
        # Restrict to under 23 hours
        route_stop_times['hour'] = route_stop_times['arrival_time'].apply(lambda x: x[start_index:end_index]).astype(int)
        route_stop_times = route_stop_times[route_stop_times['hour'] < 23]
        
        route_dict[route] = stop_route(route_stop_times.set_index('arrival_time')[['trip_id','route_id','stop_id']].to_dict('index'))
        
    stop_classes[stop_id] = stop_object(route_dict, connecting_stops, stop_to_stop_walk_time[stop_id])

    num_processed += 1
    if num_processed % 10 == 0:
        print('Progress : {}'.format((num_processed / num_stops)* 100))

f = open('Outputs/stops_dict.txt', 'wb')
pickle.dump(stop_classes,f)
f.close()

Progress : 0.47732696897374705
Progress : 0.9546539379474941
Progress : 1.431980906921241
Progress : 1.9093078758949882
Progress : 2.386634844868735
Progress : 2.863961813842482
Progress : 3.341288782816229
Progress : 3.8186157517899764
Progress : 4.295942720763723
Progress : 4.77326968973747
Progress : 5.250596658711217
Progress : 5.727923627684964
Progress : 6.205250596658711
Progress : 6.682577565632458
Progress : 7.159904534606206
Progress : 7.637231503579953
Progress : 8.1145584725537
Progress : 8.591885441527445
Progress : 9.069212410501192
Progress : 9.54653937947494
Progress : 10.023866348448687
Progress : 10.501193317422434
Progress : 10.978520286396181
Progress : 11.455847255369928
Progress : 11.933174224343675
Progress : 12.410501193317423
Progress : 12.887828162291171
Progress : 13.365155131264917
Progress : 13.842482100238662
Progress : 14.319809069212411
Progress : 14.797136038186157
Progress : 15.274463007159905
Progress : 15.75178997613365
Progress : 16.2291169451074
Pr